Il est très courant de partir d'un fichier trouvé sur la Toile **pour le convertir dans un autre format**. Ce carnet revient sur un script de conversion codé à l'occasion <a href="https://www.rue89strasbourg.com/legislatives-2017-resultats-t2-alsace-122027" target="_blank">d'une pige pour Rue89 Strasbourg</a>.

![Les législatives 2019 en Alsace (Rue89 Strasbourg](illustrations/legislatives_alsace_2019.png)

La carte étant codée avec la bibliothèque JavaScript D3, cette recette crée **un json à partir d'un XML** mis à jour au fur et à mesure du dépouillement.

Le cas des législatives est intéressant car **le nombre de candidats varie selon les circonscriptions**. Une solution assez efficace consiste à tous les ranger dans un même objet. Le script codant la carte l'utilisera ensuite pour colorer et afficher les résultats au survol. Mieux encore, on pourra le trier dans l'ordre décroissant directement à la création du json.

Il faudra également prévoir : 
- la construction d'un identifiant propre à chaque zone (dans le cas des circonscriptions, un code INSEE peut par exemple faire l'affaire dans la plupart des cas)
- le calcul direct des pourcentages (inscrits et exprimés)

On commence comme de coutume par importer les modules utiles...

In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
from urllib.request import urlopen
from lxml import etree
import json

...et déclarer les variables globales :

In [2]:
departements = ["67", "68"] # à utiliser dans une boucle for
exceptions = ["68224", "68006", "67482", "67372"] # certaines communes sont divisées entre plusieurs circo... on en tient compte pour des identifiants spéciaux
partis = {"MDM": "MODEM","COM": "PC", "SOC" : "PS", "REM" : "LREM", "REG": "UL"} # simple table de correspondance pour reformuler certaines étiquettes
liste_objets = [] # on remplira cette variable au fur et à mesure... elle deviendra le tableau d'objets final

Pour utiliser correctement lxml, il faut imaginer **une racine depuis laquelle on remonte** vers des feuilles intéressantes. A chaque branche charnière, on récupère les informations qui nous intéressent. 

Évidemment, ça ne se fait pas au doigt mouillé, et la lecture des schémas XML (comme ceux fournis par le Ministére de l'Intérieur) fait gagner énormément de temps dans le paramétrage.

Voici le gros morceau du script :

In [3]:
for departement in departements: # à chaque département, on déroule !
	arbre = etree.parse(urlopen("https://elections.interieur.gouv.fr/telechargements/LG2017/resultatsT2/0"+departement+"/0"+departement+"com.xml")) # et on interroge l'URL correspondante
	print ("Department numero "+departement)
	for noeud in arbre.xpath("//Election/Departement/Communes/Commune"): # première branche charnière
		objet = {} # depuis laquelle on crée un dictionnaire (ou objet en JS) qui sera ajouté à liste_objets
		for circo in noeud.xpath("CodCirLg"):
			codecirco = circo.text[1] # chaque circo commence par "0", on ne le considère donc pas
		for insee in noeud.xpath("CodSubCom"):
			code_insee = departement+insee.text
			if (code_insee in exceptions):
				objet["insee"] = code_insee+codecirco # si notre ville est dans les exceptions, on lui donne un code de circo
				print(objet["insee"])
			else:
				objet["insee"] = code_insee
		for resultats in noeud.xpath("Tours/Tour[NumTour=2]"): # autre branche charnière importante, celle qui concerne le second tour (pour ce cas précis)
			candidats = [] # cette variable a vocation à devenir une liste de dictionnaires, autrement dit... un tableau d'objets en JS !
			for inscrits in resultats.xpath("Mentions/Inscrits/Nombre"):
				objet["ins"] = int(inscrits.text) # int permet de convertir une chaîne de cara en entier (si elle est au bont format)
			for abstentions in resultats.xpath("Mentions/Abstentions/Nombre"):
				objet["abs"] = int(abstentions.text)
			for exprimes in resultats.xpath("Mentions/Exprimes/Nombre"):
				objet["exp"] = int(exprimes.text)
			for candidat in resultats.xpath("Resultats/Candidats/Candidat"): # et dernière branche charnière
				res_candidat = {} # pour chaque candidat, on récupère des infos avant d'ajouter ce dictionnaire dans la variable candidats
				for famille in candidat.xpath("NomPsn"):
					nfamille = famille.text
				for prenom in candidat.xpath("PrenomPsn"):
					preno = prenom.text
				res_candidat["nom"] = preno+" "+nfamille.title()
				for codenu in candidat.xpath("CodNua"):
					nunu = codenu.text
					if nunu in partis.keys(): # si la nuance récupérée fait partie des acronymes douteux...
						nunu=partis[nunu]
					res_candidat["nuance"] = nunu # ...on la remplace par l'étiquette usuelle
				for voix in candidat.xpath("NbVoix"):
					if voix == "": # on teste toujours le cas où le nombre de voix n'existerait pas
						vox = 0
					else:
						vox = int(voix.text)
				res_candidat["voix"] = vox
				res_candidat["pexp"] = round((float(vox/objet["exp"]))*100,2)
				res_candidat["pins"] = round((float(vox/objet["ins"]))*100,2)
				candidats.append(res_candidat)
				candidats = sorted(candidats, key=lambda d: d['voix'], reverse=True) # on trie dans l'ordre décroissant le tableau selon la clé "voix" de ses objets
		objet["candidats"] = candidats
		liste_objets.append(objet) # et on entérine notre variable globale

Department numero 67
673727
673728
674821
674822
674823
Department numero 68
680063
680064
682245
682246


Il ne reste plus qu'à enregistrer un json à partir de la variable liste_objets, et le tour est joué :

In [4]:
fichier = open('livraisons/leg017_communes_alsace.json','w+')
fichier.write(json.dumps(liste_objets))

220769